Import the data :

mongoimport --type csv -d energy_data -c renewables --headerline --drop MER_T10_01.csv

In [1]:
from pymongo import MongoClient

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'energy_data', 'epa', 'fruitsDB', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [4]:
# assign the database to a variable name
db = mongo['energy_data']

In [5]:
# review the collections in thedatabase
print(db.list_collection_names())

['renewables']


In [9]:
# review a document in the collection
print(db.renewables.find_one())

{'_id': ObjectId('65b30793885056d94cefd6c7'), 'MSN': 'WDPRBUS', 'YYYYMM': 194913, 'Value': 1549.262, 'Column_Order': 1, 'Description': 'Wood Energy Production', 'Unit': 'Trillion Btu'}


In [10]:
# assign the collection to a variable
renewables = db['renewables']